In [1]:
from datetime import datetime, timedelta

def calculate_workload_index(worker, max_weight, max_priority, max_time_since_due_days):
    current_time = datetime.now()  # Get current time
    workload_index = 0
    
    for task in worker['tasks']:
        weight = task['weight']
        due_date = task['due_date']
        priority = task.get('priority', 1)  # Default priority is 1 if not provided
        
        time_since_due = current_time - due_date  # Calculate time passed since the due date
        time_until_due = due_date - current_time  # Calculate time remaining until the due date
        
        if time_since_due > timedelta(0):
            overdue_factor = (1 + time_since_due.days ** 2)  # Increasing impact of overdue tasks exponentially
        else:
            overdue_factor = 1  # Handle tasks that are not overdue
        
        if time_until_due > timedelta(0):
            remaining_time_factor = 1 / (1 + time_until_due.days)  # Decreasing function for remaining time until due date based on days
        else:
            remaining_time_factor = 1  # Handle tasks that are overdue or due immediately
        
        time_factor = overdue_factor * remaining_time_factor  # Combined time factor
        
        weighted_workload = weight * time_factor * priority  # Adjusted workload based on task importance, time factor, and priority
        workload_index += weighted_workload  # Accumulate workload index for the worker


    max_time_factor = 1 + max_time_since_due_days **2 
    workload_index /= (max_weight*max_priority*max_time_factor )/100 # Normalize workload index based on worker capacity
    
    return workload_index  # Return calculated workload index


def assign_tasks_to_workers(workers, new_tasks):
    # Sort new tasks by due date (ascending order)
    new_tasks.sort(key=lambda x: x['due_date'])
    
    for new_task in new_tasks:
        # Calculate current workload index and available capacity for each worker
        for worker in workers:
            worker['workload_index'] = calculate_workload_index(worker,max_weight, max_time_since_due, max_priority)

        # Find the worker with the lowest workload index
        min_workload_worker = min(workers, key=lambda w: w['workload_index'])
        
        # Assign the new task to this worker
        min_workload_worker['tasks'].append(new_task)
        
        # Recalculate the workload index and available capacity for this worker only
        min_workload_worker['workload_index'] = calculate_workload_index(min_workload_worker,max_weight, max_time_since_due, max_priority)

    return workers


# Example usage

# Sample data
tasks1 = [
    {'weight': 6, 'due_date': datetime.now() + timedelta(days=1), 'priority': 2},
    {'weight': 7, 'due_date': datetime.now() - timedelta(days=1), 'priority': 1}
]
tasks2 = [
    {'weight': 8, 'due_date': datetime.now() + timedelta(days=3), 'priority': 1},
    {'weight': 5, 'due_date': datetime.now() - timedelta(days=2), 'priority': 3}
]

workers = [
    {'tasks': tasks1,'workload_index': 0, },  
    {'tasks': tasks2,'workload_index': 0, }   
]

new_tasks = [
    {'weight': 10, 'due_date': datetime.now() + timedelta(days=6), 'priority': 2},
    {'weight': 6, 'due_date': datetime.now() + timedelta(days=7), 'priority': 1},
    {'weight': 12, 'due_date': datetime.now() + timedelta(days=1), 'priority': 3}
]



max_weight = 30 # max_working_day that a process can have   
max_time_since_due = 30 # max_time_since_due in planning horizon
max_priority = 3 # max_priority possibler
for i, worker in enumerate(workers): 
    worker["workload_index"] = calculate_workload_index(worker, max_weight, max_time_since_due, max_priority)
    print(f"Worker {i+1} workload index: {worker['workload_index']}")

# Assign the new tasks to the workers with the lowest workload index
updated_workers = assign_tasks_to_workers(workers, new_tasks)

# Print results
for i, worker in enumerate(updated_workers):
    print(f"Worker {i+1} workload index: {worker['workload_index']}")
    for task in worker['tasks']:
        print(f" - Task: weight={task['weight']}, due_date={task['due_date']}, priority={task['priority']}")



Worker 1 workload index: 0.28888888888888886
Worker 2 workload index: 0.862962962962963
Worker 1 workload index: 0.7354497354497355
 - Task: weight=6, due_date=2024-07-02 21:44:19.110083, priority=2
 - Task: weight=7, due_date=2024-06-30 21:44:19.110083, priority=1
 - Task: weight=12, due_date=2024-07-02 21:44:19.110083, priority=3
 - Task: weight=10, due_date=2024-07-07 21:44:19.110083, priority=2
 - Task: weight=6, due_date=2024-07-08 21:44:19.110083, priority=1
Worker 2 workload index: 0.862962962962963
 - Task: weight=8, due_date=2024-07-04 21:44:19.110083, priority=1
 - Task: weight=5, due_date=2024-06-29 21:44:19.110083, priority=3
